In [18]:
import datetime as dt

import xarray as xr

from mcs_prime import PATHS, McsTracks, PixelData

year = 2019
month = 6
day = 21
e5datadir = PATHS['era5dir'] / f'data/oper/an_sfc/{year}/{month:02d}/{day:02d}'

h = 6

e5time = dt.datetime(year, month, day, h, 0)
paths = [e5datadir / (f'ecmwf-era5_oper_an_sfc_{t.year}{t.month:02d}{t.day:02d}'
                      f'{t.hour:02d}00.{var}.nc')
         for var in ['cape', 'tcwv']
         for t in [e5time, e5time + dt.timedelta(hours=1)]]

# Similar to Li 2023 region but slightly larger.
lat_slice = slice(55, 0)
lon_slice = slice(95, 165)


e5data = (xr.open_mfdataset(paths).sel(latitude=lat_slice, longitude=lon_slice)
          .interp(time=e5time + dt.timedelta(minutes=30)).load())


e5data

<xarray.Dataset>
Dimensions:    (longitude: 281, latitude: 221)
Coordinates:
  * longitude  (longitude) float32 95.0 95.25 95.5 95.75 ... 164.5 164.8 165.0
  * latitude   (latitude) float32 55.0 54.75 54.5 54.25 ... 0.75 0.5 0.25 0.0
    time       datetime64[ns] 2019-06-21T06:30:00
Data variables:
    cape       (latitude, longitude) float64 21.53 0.0001221 ... 121.5 109.6
    tcwv       (latitude, longitude) float64 18.12 18.03 17.84 ... 61.72 61.13
Attributes:
    Conventions:  CF-1.6
    history:      2019-09-09 11:04:25 GMT by grib_to_netcdf-2.12.5: grib_to_n...

In [19]:
outdir = PATHS['outdir'] / 'regional_ERA5_data' / f'{e5time.year}' / f'{e5time.month:02d}' / f'{e5time.day:02d}'
filename = f'ecmwf-era5_oper_an_sfc_{e5time.year}{e5time.month:02d}{e5time.day:02d}{e5time.hour:02d}30.east_asia.nc'
outpath = outdir / filename
outpath

PosixPath('/gws/nopw/j04/mcs_prime/mmuetz/data/mcs_prime_output/regional_ERA5_data/2019/06/21/ecmwf-era5_oper_an_sfc_201906210630.east_asia.nc')

In [20]:
# WILL NOT RUN IN NOTEBOOK:
# Has no write perms. Run on JASMIN instead.
# outpath.parent.mkdir(exist_ok=True, parents=True)
# e5data.to_netcdf(outpath)

In [21]:
ds = xr.open_dataset(outpath)
ds

<xarray.Dataset>
Dimensions:    (longitude: 281, latitude: 221)
Coordinates:
  * longitude  (longitude) float32 95.0 95.25 95.5 95.75 ... 164.5 164.8 165.0
  * latitude   (latitude) float32 55.0 54.75 54.5 54.25 ... 0.75 0.5 0.25 0.0
    time       datetime64[ns] ...
Data variables:
    cape       (latitude, longitude) float64 ...
    tcwv       (latitude, longitude) float64 ...
Attributes:
    Conventions:  CF-1.6
    history:      2019-09-09 11:04:25 GMT by grib_to_netcdf-2.12.5: grib_to_n...

In [22]:
ds.time

<xarray.DataArray 'time' ()>
array('2019-06-21T06:30:00.000000000', dtype='datetime64[ns]')
Coordinates:
    time     datetime64[ns] 2019-06-21T06:30:00
Attributes:
    long_name:  time